In [55]:
from dateutil.relativedelta import relativedelta
from dateutil.utils import today
from eolearn.core import FeatureType, SaveTask, linearly_connect_tasks, EOWorkflow, EOExecutor
from eolearn.io import get_available_timestamps, SentinelHubInputTask
from sentinelhub import SHConfig, BBox, CRS, DataCollection
from matplotlib import pyplot as plt


In [56]:
config = SHConfig(profile="sentinel-dl")

In [57]:
roi_bbox = BBox(bbox=(5.60, 52.68, 5.75, 52.63), crs=CRS.WGS84)

In [58]:
def get_last_two_timestamps(data_collection):
    today_ts = today()
    two_months_ago_ts = today_ts - relativedelta(months=2)
    time_of_interest = (two_months_ago_ts, today_ts)
    
    timestamps = get_available_timestamps(bbox=roi_bbox, time_interval=time_of_interest, data_collection=data_collection,  config=config)
    return timestamps[-2:]

In [59]:
# max cloud cover
maxcc = 0.8

# meters
resolution = 20

data_collection = DataCollection.SENTINEL2_L1C

time_interval = get_last_two_timestamps(data_collection)

input_task = SentinelHubInputTask(
    data_collection=data_collection,
    # bands=["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B10", "B11", "B12"] # if not passed, it will download all
    bands_feature=(FeatureType.DATA, "L1C_data"),
    additional_data=[(FeatureType.MASK, "dataMask")],   # cloud mask
    maxcc=maxcc, 
    resolution=resolution,
    config=config,  # important since we are using sentinel-dl, alternatively save ID and secret to default profile
)

besides L1C we can also get L2A, SCL, NDWI... Refer to this guide: https://github.com/sentinel-hub/eo-learn/blob/master/examples/io/SentinelHubIO.ipynb

In [60]:
save = SaveTask("eopatch_fd")

In [68]:
workflow_nodes = linearly_connect_tasks(input_task, save)
workflow = EOWorkflow(workflow_nodes)
# workflow.dependency_graph() additionally install graphviz and eo-learn[VISUALIZATION]

In [69]:
# take nodes, these are not the same as tasks!!!
input_node = workflow_nodes[0]
save_node = workflow_nodes[-1]

execution_args = [{
        input_node: {"bbox": roi_bbox, "time_interval": time_interval},
        save_node: {"eopatch_folder": "eopatch_file"}
}]

executor = EOExecutor(workflow, execution_args)

In [70]:
executor.run(workers=4)

executor.make_report()

failed_ids = executor.get_failed_executions()
if failed_ids:
    raise RuntimeError(
        f"Execution failed EOPatches with IDs:\n{failed_ids}\n"
        f"For more info check report at {executor.get_report_path()}"
    )

100%|██████████| 1/1 [00:05<00:00,  5.50s/it]
C:\Users\blazr\miniconda3\envs\sentinel_env\lib\site-packages\eolearn\visualization\eoexecutor.py:62: EOUserWarning: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH.
Please install the system package 'graphviz' (in addition to the python package) to have the dependency graph in the final report!
  warnings.warn(
